# A Different Method for the Tracking Script

In this notebook, I'm going to work on writing a different method that might be a little faster. Making an individual API call for each package simplifies the code, but I bet it'll be faster if I do group calls. You can only do up to 35, so I'll need to figure out how to handle the list while it's longer than 35.

In [2]:
# OK, let's see if I remember how modulus works...
115 % 35

10

In [5]:
# OK, how about this?
print(115/35)
int(115/35)

3.2857142857142856


3

In [7]:
# Does it always round down?
int(69/35)

1

In [3]:
import pandas as pd
tracking_df = pd.read_csv('TrackingList.csv')
tracking_df.tail()

,Tracking Nos.,Client Nos.,Comments
113,9114902200852144849386,50836.02,1040 and Source Docs
114,9114902200852144849393,54159.02,Source Docs
115,9114902200852144849416,70250.02,Source Docs
116,9114902200852144849409,63909.02,Source Docs and EL
117,9114902200852144849423,14246.02,Form 1041; Attempt 2


In [5]:
track_list = list(tracking_df['Tracking Nos.'])

OK, let's think about how we need to slice this. In this case, we need a list for 1 through 35, 36 through 70, 71 through 105, and 106 through 117.
For python, that'll be:
 - range(35), for 0 through 34
 - range(35,70), for 35 through 69
 - range(70,105), for 70 through 104
 - range(105,117), for 105 through 116

So here's the pattern:
 - let length = len(track_list)
 - batches = int(len(track_list) / 35)
 - list[35n : 35 * (n + 1)] for 0 to batches
 - i = length - (lenth % 35) (so this time, it'll give us 105)
 - list[i:(length)]

In [7]:
# OK, first we want to break track_list up into lists of 35 or fewer
length = len(track_list)
fullbatches = int(length/35)
batches = fullbatches + 1

for i in range(fullbatches):
    print(track_list[35*i : 35*(i+1)])

print(track_list[35*fullbatches : length])

['70191640000164106422', '70192280000060192481', '9114902200789659810412', '9114902200789659710238', '9114902200789659810320', '9114902200789659810290', '9114902200789659810306', '9114902200852116737802', '9114902200852453171765', '70211970000045308191', '70211970000045307835', '70211970000045309723', '70211970000045308061', '70211970000045308856', '70211970000045307965', '70191640000164106507', '70192280000060192474', '70211970000045308634', '70191640000164105357', '70192280000060191101', '70192280000060191941', '70196140000164105869', '70191640000164105913', '70191640000164105265', '70192280000060190616', '70192280000060190173', '70191640000164105647', '70211970000045308283', '70191640000164105234', '70211970000045308269', '70191640000164105319', '70191640000164106705', '70191640000164106569', '70191640000164106583', '70191640000164106545']
['70211970000045308634', '70191640000164106552', '70191640000164106477', '70191640000164106484', '70191640000164106521', '70191640000164106538', 

In [9]:
for i in range(fullbatches):
    print(len(track_list[35*i : 35*(i+1)]))

print(len(track_list[35*fullbatches : length]))
print(length)

35
35
35
13
118


In [10]:
# So we could do:
import requests
from config import userid, password
from bs4 import BeautifulSoup

# Initialize some stuff:
base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='
delivered_list = []

# Build the URL:
for i in range(fullbatches):
    batchlist = track_list[35*i : 35*(i+1)]
    call_url = base_url + f'<TrackRequest USERID="{userid}" PASSWORD="{password}">'
    for number in batchlist:
        call_url += f'<TrackID ID="{number}"></TrackID>'
    call_url += '</TrackRequest>'

    # Make the API call and save the response:    
    response = requests.get(call_url)
    soup = BeautifulSoup(response.content, 'xml')
    # TODO: Read the xml for this batch as in the cells below
    
## YOU'RE GOING TO HAVE TO READ THE XML MORE CAREFULLY THIS TIME. ##
    
# Three different phrases might indicate that the package has reached the recipient:
# condition1 = 'Your item was delivered' in str(response.content)
# condition2 = 'Your item has been delivered' in str(response.content)
# condition3 = 'Your item was picked up' in str(response.content)

# if condition1 | condition2 | condition3:
#     delivered_list.append(num)
#     spots = tracking_df[tracking_df['Tracking Nos.'] == num].index
#     # In most cases, there will only be one spot in spots.
#     for spot in spots:
#         deliveredfile.write(f'\n{tracking_df.iloc[spot,0]}, {tracking_df.iloc[spot,1]}, {tracking_df.iloc[spot,2]}')
#         deliveredfile.write('\n\n' + soup.TrackSummary.text + '\n')

In [27]:
# Let's do an api call for a short list of packages and see about reading the response

exp_list = [
    9114902200852144849218,
    9114902200852144849393,
    9114902200852144849386
]
# The first one has been delivered; the other two haven't

exp_url = base_url + f'<TrackRequest USERID="{userid}" PASSWORD="{password}">'
for number in exp_list:
    exp_url += f'<TrackID ID="{number}"></TrackID>'
exp_url += '</TrackRequest>'
response = requests.get(exp_url)
soup = BeautifulSoup(response.content, 'xml')

soup

<?xml version="1.0" encoding="utf-8"?>
<TrackResponse><TrackInfo ID="9114902200852144849218"><TrackSummary>Your item was delivered in or at the mailbox at 12:28 pm on July 29, 2024 in DETROIT, MI 48221.</TrackSummary><TrackDetail>Arrived at USPS Regional Facility, July 27, 2024, 1:07 pm, DETROIT MI DISTRIBUTION CENTER</TrackDetail><TrackDetail>In Transit to Next Facility, 07/26/2024</TrackDetail><TrackDetail>Arrived at USPS Regional Origin Facility, 07/25/2024, 2:52 pm, COPPELL TX DISTRIBUTION CENTER</TrackDetail><TrackDetail>USPS picked up item, July 23, 2024, 3:26 pm, WACO, TX 76710</TrackDetail></TrackInfo><TrackInfo ID="9114902200852144849393"><TrackSummary>Your item arrived at our FORT WORTH TX DISTRIBUTION CENTER origin facility on July 29, 2024 at 11:26 pm. The item is currently in transit to the destination.</TrackSummary><TrackDetail>USPS picked up item, July 29, 2024, 2:47 pm, WACO, TX 76710</TrackDetail></TrackInfo><TrackInfo ID="9114902200852144849386"><TrackSummary>Your it

In [26]:
# Same output as soup.find_all('TrackSummary'), but will integrate with the loop a little better:
for number in exp_list:
    print(soup.find('TrackInfo', ID=number).find('TrackSummary'))

<TrackSummary>Your item was delivered in or at the mailbox at 12:28 pm on July 29, 2024 in DETROIT, MI 48221.</TrackSummary>
<TrackSummary>Your item arrived at our FORT WORTH TX DISTRIBUTION CENTER origin facility on July 29, 2024 at 11:26 pm. The item is currently in transit to the destination.</TrackSummary>
<TrackSummary>Your item departed our USPS facility in AUSTIN TX DISTRIBUTION CENTER on July 30, 2024 at 2:18 pm. The item is currently in transit to the destination.</TrackSummary>


In [28]:
delivered_list = []
# TODO: Open the 'deliveredfile' here

for number in exp_list:
    summary = soup.find('TrackInfo', ID=number).find('TrackSummary')

    # Three different phrases might indicate that the package has reached the recipient:
    condition1 = 'Your item was delivered' in summary.text
    condition2 = 'Your item has been delivered' in summary.text
    condition3 = 'Your item was picked up' in summary.text

    if condition1 | condition2 | condition3:
        delivered_list.append(number)
        # TODO: Add number, client number, comments, summary, to the delivered file

delivered_list

[9114902200852144849218]

In [2]:
import pandas as pd
a = {1:'first',2:'second',3:'third'}
b = {1:'apple',2:'banana',3:'cantaloupe'}
c = {1:'avocado',2:'beet',3:'carrot'}
df = pd.DataFrame([a,b,c])
df

,1,2,3
0,first,second,third
1,apple,banana,cantaloupe
2,avocado,beet,carrot


In [13]:
spot = df[df[1] == 'avocado'].index[0]
spot

2

In [4]:
import datetime as dt
rn = dt.datetime.now()
thing = dt.datetime.strftime(rn, '%d %B %Y %H:%M')
thing

'01 August 2024 13:47'